# Batch Legal – BERT-Model-Pipe: Data Retrieval – Preprocessing – Modelling- Dictionaries – Plotly-Function for Streamlit




## Imports for the entire Notebook

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet -U spacy

In [ ]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.2 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!pip install spacy-lookups-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Imports

import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.collocations import *

import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


## Data Retrieval and Exploration

In [ ]:
#Jakob, you should enter your code here!

## Preprocessing

In [4]:
#Loading a lot of data from csv
"""CSV with over 5000 documents"""

data = pd.read_csv("/content/drive/MyDrive/over_5000_docs_scraped.csv")

In [ ]:
#Starting the actual Preprocessing
df_content = data.Content

In [ ]:
#Preproc-Function

def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercasing 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    tokenized_sentence = word_tokenize(sentence) ## tokenizing 
    stop_words = set(stopwords.words('english')) ## defining stopwords
    tokenized_sentence_cleaned = [w for w in tokenized_sentence 
                                  if not w in stop_words] ## remove stopwords
    
    #tokenized_sentence_cleaned = [w for w in tokenized_sentence_cleaned if not w in ignore_list] COMMENTED IGNORE OUT!
    
    sentences = ' '.join(word for word in tokenized_sentence_cleaned)
    
    #spacy
    nlp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler"])
    nlp.remove_pipe("lemmatizer")
    nlp.add_pipe("lemmatizer", config={"mode": "lookup"}).initialize()
    doc = nlp(sentences)
    lemmatized = " ".join([token.lemma_ for token in doc])
    
    return lemmatized

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Applying Cleaning Function

clean_txt = df_content.apply(cleaning)

In [ ]:
len(clean_txt)

5147

In [ ]:
clean_txt_df = pd.DataFrame(clean_txt)

In [ ]:
#Save cleaned txt as csv

clean_txt_df.to_csv('/content/drive/MyDrive/5000_docs_clean_txt.csv')

In [146]:
clean_txt_df = pd.read_csv('/content/drive/MyDrive/5000_docs_clean_txt.csv')

In [147]:
clean_txt_df

,Unnamed: 0,Content
0,0,financial crisis expose weakness transparency ...
1,1,condition make fertiliser available internal m...
2,2,regulation ec aim establish maintain high unif...
3,3,council regulation ec substantially amend seve...
4,4,regulation ec provide authorisation additive u...
...,...,...
5142,5142,commission regulation ec lay detail rule imple...
5143,5143,article regulation eu establish criterium gran...
5144,5144,annex regulation eu establish union list subst...
5145,5145,commission implement regulation eu provide ope...


In [148]:
clean_txt = clean_txt_df['Content']

In [ ]:
all_content = ''.join(clean_txt)

In [ ]:
from collections import Counter

# split() returns list of all the words in the string
split_it = all_content.split()

# Pass the split_it list to instance of Counter class.
Counters_found = Counter(split_it)
#print(Counters)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counters_found.most_common(100)


In [ ]:
most_occur

[('shall', 170376),
 ('regulation', 145856),
 ('article', 116582),
 ('—', 89977),
 ('union', 78524),
 ('–', 77468),
 ('commission', 74378),
 ('’', 71488),
 ('state', 70478),
 ('member', 67881),
 ('eu', 65644),
 ('use', 60345),
 ('‘', 55578),
 ('product', 52590),
 ('provide', 50942),
 ('accordance', 50942),
 ('authority', 49372),
 ('follow', 49061),
 ('information', 46957),
 ('refer', 45008),
 ('b', 43241),
 ('include', 43190),
 ('may', 39217),
 ('point', 37340),
 ('measure', 36308),
 ('datum', 36005),
 ('annex', 35252),
 ('european', 34320),
 ('period', 32989),
 ('market', 32881),
 ('set', 32731),
 ('c', 30691),
 ('concern', 29547),
 ('apply', 29487),
 ('ec', 28902),
 ('system', 28141),
 ('take', 27709),
 ('producer', 26439),
 ('import', 26028),
 ('price', 25974),
 ('paragraph', 25896),
 ('report', 25860),
 ('export', 25292),
 ('requirement', 25037),
 ('mean', 25011),
 ('country', 24292),
 ('application', 24057),
 ('relevant', 23636),
 ('establish', 23137),
 ('within', 23103),
 ('make'

In [ ]:
# list used to remove 10 most frequent words + 'european'

ignore_list = ['shall', 'regulation', 'article', 'union', 'state', 'eu', 'official',  'member', 'commission', 'commission', 'accordance', 'european',
              'imo', 'iec', 'reg', 'solas', 'resmsc', 'hsc', 'ed', 'res', 'incl', 'corr', 'msccirc', 'msc']

In [ ]:
#Function to get rid of these terms

def ignore(sentence):
  tokenized_sentence = word_tokenize(sentence) ## tokenizing 
  cleaned  = [w for w in tokenized_sentence if not w in ignore_list]
  sentence_cleaned = ' '.join(word for word in cleaned)
  return sentence_cleaned

In [ ]:
#Applying function

txt_clean = clean_txt.apply(ignore)

In [ ]:
txt_clean.head()

0    financial crisis expose weakness transparency ...
1    condition make fertiliser available internal m...
2    ec aim establish maintain high uniform level c...
3    council ec substantially amend several time in...
4    ec provide authorisation additive use animal n...
Name: Content, dtype: object

In [ ]:
#Saving txt_clean

txt_clean.to_csv('/content/drive/MyDrive/5000_txt_clean.csv')

In [5]:
#Loading txt_clean

txt_clean = pd.read_csv('/content/drive/MyDrive/5000_docs_clean_txt.csv')

In [4]:
txt_clean

,Unnamed: 0,Content
0,0,financial crisis expose weakness transparency ...
1,1,condition make fertiliser available internal m...
2,2,regulation ec aim establish maintain high unif...
3,3,council regulation ec substantially amend seve...
4,4,regulation ec provide authorisation additive u...
...,...,...
5142,5142,commission regulation ec lay detail rule imple...
5143,5143,article regulation eu establish criterium gran...
5144,5144,annex regulation eu establish union list subst...
5145,5145,commission implement regulation eu provide ope...


In [6]:
data.Content = txt_clean['Content']

In [157]:
data.head()

,Unnamed: 0,title,cellar,date,dir_code,dir_1,dir_2,dir_3,dir_4,dir_5,dir_6,Content
0,0,Regulation (EU) No 600/2014 of the European Pa...,3b729ddf-f1f7-11e3-8cd4-01aa75ed71a1,2014-06-12,6202025.0,Right of establishment and freedom to provide ...,Sectoral application,Service activities,Stock exchanges and other securities markets,NaN,NaN,financial crisis expose weakness transparency ...
1,1,Regulation (EU) 2019/1009 of the European Parl...,e351eb07-9713-11e9-9369-01aa75ed71a1,2019-06-25,133019.0,Industrial policy and internal market,Internal market: approximation of laws,Fertilisers,NaN,NaN,NaN,condition make fertiliser available internal m...
2,2,Commission Regulation (EU) No 139/2014 of 12 F...,3cc00bdc-955e-11e3-8c34-01aa75ed71a1,2014-02-14,74030.0,Transport policy,Air transport,Air safety,NaN,NaN,NaN,regulation ec aim establish maintain high unif...
3,3,Regulation (EU) 2018/196 of the European Parli...,fa9532bb-12e6-11e8-9253-01aa75ed71a1,2018-02-16,2303040.0,Customs Union and free movement of goods,Application of the Common Customs Tariff,Tariff derogations,Reintroduction of customs duties,NaN,NaN,council regulation ec substantially amend seve...
4,4,Commission Implementing Regulation (EU) 2021/4...,61d76918-8b7a-11eb-b85c-01aa75ed71a1,2021-03-23,35010.0,Agriculture,Approximation of laws and health measures,Animal feedingstuffs,NaN,NaN,NaN,regulation ec provide authorisation additive u...


In [ ]:
#Transforming Series in List

txt_clean = txt_clean.Content.tolist()

# Topic-Modelling with BERTopic




In [8]:
#PIP-installing BERTtopic

!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 5.0 MB/s 
     |████████████████████████████████| 5.2 MB 29.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 7.2 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 4.2 MB 59.8 MB/s 
     |████████████████████████████████| 1.2 MB 54.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 636 kB 85.2 MB/s 
     |████████████████████████████████| 6.6 MB 64.7 MB/s 
     |████████████████████████████████| 1.1 MB 51.8 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342189 sha256=dfdfea3df1df461733892f528d1c4920b87d337ba913c793f91437530baf1f4e
  Stored in directory: /root/.cache/p

In [9]:
from bertopic import BERTopic #BERTtopic-model: https://github.com/MaartenGr/BERTopic

# Topic-Modelling Entire Data

### BERTopic-model

In [ ]:
#Training

topic_model_all = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics_all, probs = topic_model_all.fit_transform(txt_clean)

Batches:   0%|          | 0/161 [00:00<?, ?it/s]

2022-06-07 12:57:50,647 - BERTopic - Transformed documents to Embeddings
2022-06-07 12:58:19,114 - BERTopic - Reduced dimensionality
2022-06-07 12:58:22,417 - BERTopic - Clustered reduced embeddings


In [ ]:
topic_model_all.get_topic_info()

,Topic,Count,Name
0,-1,837,-1_follow_design_provide_use
1,0,243,0_institution_exposure_resolution_credit
2,1,141,1_biocidal_producttype_substance_active
3,2,119,2_fish_vessel_catch_fishery
4,3,78,3_food_novel_infant_applicant
...,...,...,...
131,130,11,130_rough_diamond_kimberley_kp
132,131,11,131_register_name_di_enter
133,132,11,132_additive_formaldehyde_fee_diclazuril
134,133,11,133_register_enter_name_journal


In [ ]:
topic_model_all.visualize_barchart()

In [ ]:
topic_model_all.get_topics()

In [ ]:
topic_model_all.visualize_topics()

In [ ]:
topic_model_all.visualize_hierarchy()

### LDA-Model

In [ ]:
# bigram vectorization

vectorizer_n_gram = TfidfVectorizer(ngram_range = (1,1)) # BI-GRAMS
cleaned_vectorizer_n_gram = vectorizer_n_gram.fit_transform(txt_clean)

In [ ]:
df_animal = pd.DataFrame(cleaned_vectorizer_n_gram.toarray(), columns=vectorizer_n_gram.get_feature_names_out())

In [ ]:
#Topic model function from ML-10-lecture
def print_topics(model, vectorizer, top_words):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print([(vectorizer_n_gram.get_feature_names_out()[i], round(topic[i],2))
                        for i in topic.argsort()[:-top_words - 1:-1]])

In [ ]:
# Instantiating the LDA 
n_components = 10
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)

# Fitting the LDA on the vectorized documents
lda_model.fit(df_animal)

LatentDirichletAllocation(max_iter=100, n_components=2)

In [ ]:
print_topics(lda_model, cleaned_vectorizer_n_gram, top_words=20)

--------------------
Topic 0:
[('additive', 154.85), ('ec', 152.87), ('use', 143.37), ('product', 133.5), ('annex', 130.22), ('provide', 128.76), ('authority', 123.43), ('information', 116.81), ('measure', 114.51), ('set', 107.09), ('column', 104.06), ('import', 99.36), ('substance', 95.04), ('follow', 93.17), ('may', 93.11), ('good', 92.79), ('period', 89.33), ('application', 89.15), ('animal', 87.43), ('apply', 86.52)]
--------------------
Topic 1:
[('import', 151.91), ('value', 144.45), ('standard', 143.94), ('amendment', 131.03), ('implement', 111.66), ('fix', 106.31), ('name', 103.42), ('journal', 98.58), ('day', 87.9), ('enter', 86.71), ('register', 85.62), ('specification', 80.07), ('uruguay', 73.6), ('xvi', 72.33), ('annex', 71.92), ('variable', 70.71), ('negotiation', 70.48), ('multilateral', 68.53), ('round', 68.4), ('stipulate', 66.57)]


#Creating a function to speed up Modelling of Sub_Directories

In [108]:
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
import numpy as np
from umap import UMAP

from typing import List
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

def model_to_figure(data, dir, name):
  df_selec = data.loc[data[f'dir_{dir}'] == name]
  txt = df_selec.Content
  txt = txt.tolist()
  
  #Training model
  #umap_model = UMAP(init='random')
  
  model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
  topics, probs = model.fit_transform(txt)

  #Building Dict
  dictionary = {"Sub_dir_name": name, "get_topic": model.get_topics(), "topic_freq": model.get_topic_freq(), "topic_sizes": model.topic_sizes}

  #Getting Embeddings for the visualize_topic_function
  # Select topics based on top_n and topics args
  freq_df = model.get_topic_freq()
  topics = sorted(freq_df.Topic.to_list())

  # Extract topic words and their frequencies
  topic_list = sorted(topics)
  frequencies = [model.topic_sizes[topic] for topic in topic_list]
  words = [" | ".join([word[0] for word in model.get_topic(topic)[:5]]) for topic in topic_list]

  # Embed c-TF-IDF into 2D
  all_topics = sorted(list(model.get_topics().keys()))
  indices = np.array([all_topics.index(topic) for topic in topics])
  if len(topics) > 1:
    embeddings_1 = model.c_tf_idf.toarray()[indices]
    embeddings = MinMaxScaler().fit_transform(embeddings_1)
    embeddings = UMAP(n_neighbors=2, n_components=2, metric='hellinger').fit_transform(embeddings)
  else:
    embeddings = []

  if len(topics) != 1:
    distance_matrix = 1 - cosine_similarity(embeddings_1)
  else:
    distance_matrix = []

  return dictionary, embeddings, distance_matrix

##Bar-Chart Function

In [11]:
import itertools
import numpy as np
from typing import List

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def bert_bar(topic_freq, get_topic,
             topics: List[int] = None,
             top_n_topics: int = 10,
             n_words: int = 5,
             width: int = 250,
             height: int = 250) -> go.Figure:
  
  
  colors = itertools.cycle(["#D55E00", "#0072B2", "#CC79A7", "#E69F00", "#56B4E9", "#009E73", "#F0E442"])
  
  # Select topics based on top_n and topics args
  freq_df = topic_freq
  #freq_df = freq_df.loc[freq_df.Topic != -1, :]
  if topics is not None:
      topics = list(topics)
  elif top_n_topics is not None:
      topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
  else:
      topics = sorted(freq_df.Topic.to_list()[0:6])
  
  print(topics)

  # Initialize figure
  subplot_titles = [f"Topic {topic}" for topic in topics]
  columns = 4
  rows = int(np.ceil(len(topics) / columns))
  fig = make_subplots(rows=rows,
                      cols=columns,
                      shared_xaxes=False,
                      horizontal_spacing=.1,
                      vertical_spacing=.4 / rows if rows > 1 else 0,
                      subplot_titles=subplot_titles)

  # Add barchart for each topic
  row = 1
  column = 1
  for topic in topics:
    words = [word + "  " for word, _ in get_topic[topic]][:n_words][::-1]
    scores = [score for _, score in get_topic[topic]][:n_words][::-1]
    
    fig.add_trace(
        go.Bar(x=scores,
                   y=words,
                   orientation='h',
                   marker_color=next(colors)),
        row=row, col=column)

    if column == columns:
        column = 1
        row += 1
    else:
        column += 1

  # Stylize graph
  fig.update_layout(
      template="plotly_white",
      showlegend=False,
      title={
            'text': "<b>Topic Word Scores",
            'x': .5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
      },
      width=width*4,
      height=height*rows if rows > 1 else height * 1.3,
      hoverlabel=dict(
          bgcolor="white",
          font_size=16,
          font_family="Rockwell"
      ),
  )

  fig.update_xaxes(showgrid=True)
  fig.update_yaxes(showgrid=True)

  return fig


##Topics Function


In [12]:
import numpy as np
import pandas as pd
from typing import List

import plotly.express as px
import plotly.graph_objects as go


def visualize_topics(topic_freq, topic_sizes, get_topic, embeddings,
                     topics: List[int] = None,
                     top_n_topics: int = None,
                     width: int = 650,
                     height: int = 650) -> go.Figure:
    """ Visualize topics, their sizes, and their corresponding words

    This visualization is highly inspired by LDAvis, a great visualization
    technique typically reserved for LDA.

    Arguments:
        topic_model: A fitted BERTopic instance.
        topics: A selection of topics to visualize
        top_n_topics: Only select the top n most frequent topics
        width: The width of the figure.
        height: The height of the figure.

    Usage:

    To visualize the topics simply run:

    ```python
    topic_model.visualize_topics()
    ```

    Or if you want to save the resulting figure:

    ```python
    fig = topic_model.visualize_topics()
    fig.write_html("path/to/file.html")
    ```
    <iframe src="../../getting_started/visualization/viz.html"
    style="width:1000px; height: 680px; border: 0px;""></iframe>
    """
    # Select topics based on top_n and topics args
    freq_df = topic_freq
    #freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        topics = sorted(freq_df.Topic.to_list())

    # Extract topic words and their frequencies
    topic_list = sorted(topics)
    frequencies = [topic_sizes[topic] for topic in topic_list]
    words = [" | ".join([word[0] for word in get_topic[topic][:5]]) for topic in topic_list]

    # Visualize with plotly
    df = pd.DataFrame({"x": embeddings[:, 0], "y": embeddings[:, 1],
                       "Topic": topic_list, "Words": words, "Size": frequencies})
    return _plotly_topic_visualization(df, topic_list, width, height)


def _plotly_topic_visualization(df: pd.DataFrame,
                                topic_list: List[str],
                                width: int,
                                height: int):
    """ Create plotly-based visualization of topics with a slider for topic selection """

    def get_color(topic_selected):
        if topic_selected == -1:
            marker_color = ["#B0BEC5" for _ in topic_list]
        else:
            marker_color = ["red" if topic == topic_selected else "#B0BEC5" for topic in topic_list]
        return [{'marker.color': [marker_color]}]

    # Prepare figure range
    x_range = (df.x.min() - abs((df.x.min()) * .15), df.x.max() + abs((df.x.max()) * .15))
    y_range = (df.y.min() - abs((df.y.min()) * .15), df.y.max() + abs((df.y.max()) * .15))

    # Plot topics
    fig = px.scatter(df, x="x", y="y", size="Size", size_max=40, template="simple_white", labels={"x": "", "y": ""},
                     hover_data={"Topic": True, "Words": True, "Size": True, "x": False, "y": False})
    fig.update_traces(marker=dict(color="#B0BEC5", line=dict(width=2, color='DarkSlateGrey')))

    # Update hover order
    fig.update_traces(hovertemplate="<br>".join(["<b>Topic %{customdata[0]}</b>",
                                                 "Words: %{customdata[1]}",
                                                 "Size: %{customdata[2]}"]))

    # Create a slider for topic selection
    steps = [dict(label=f"Topic {topic}", method="update", args=get_color(topic)) for topic in topic_list]
    sliders = [dict(active=0, pad={"t": 50}, steps=steps)]

    # Stylize layout
    fig.update_layout(
        title={
            'text': "<b>Intertopic Distance Map",
            'y': .95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        width=width,
        height=height,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
        xaxis={"visible": False},
        yaxis={"visible": False},
        sliders=sliders
    )

    # Update axes ranges
    fig.update_xaxes(range=x_range)
    fig.update_yaxes(range=y_range)

    # Add grid in a 'plus' shape
    fig.add_shape(type="line",
                  x0=sum(x_range) / 2, y0=y_range[0], x1=sum(x_range) / 2, y1=y_range[1],
                  line=dict(color="#CFD8DC", width=2))
    fig.add_shape(type="line",
                  x0=x_range[0], y0=sum(y_range) / 2, x1=x_range[1], y1=sum(y_range) / 2,
                  line=dict(color="#9E9E9E", width=2))
    fig.add_annotation(x=x_range[0], y=sum(y_range) / 2, text="D1", showarrow=False, yshift=10)
    fig.add_annotation(y=y_range[1], x=sum(x_range) / 2, text="D2", showarrow=False, xshift=10)
    fig.data = fig.data[::-1]

    return fig


## Hierachy viz

In [13]:
import numpy as np
from scipy.cluster.hierarchy import linkage
from typing import List
from sklearn.metrics.pairwise import cosine_similarity

import plotly.graph_objects as go
import plotly.figure_factory as ff


def visualize_hierarchy(topic_freq, get_topic, distance_matrix,
                        orientation: str = "left",
                        topics: List[int] = None,
                        top_n_topics: int = None,
                        width: int = 1000,
                        height: int = 600) -> go.Figure:
    """ Visualize a hierarchical structure of the topics

    A ward linkage function is used to perform the
    hierarchical clustering based on the cosine distance
    matrix between topic embeddings.

    Arguments:
        topic_model: A fitted BERTopic instance.
        orientation: The orientation of the figure.
                     Either 'left' or 'bottom'
        topics: A selection of topics to visualize
        top_n_topics: Only select the top n most frequent topics
        width: The width of the figure. Only works if orientation is set to 'left'
        height: The height of the figure. Only works if orientation is set to 'bottom'

    Returns:
        fig: A plotly figure
    """

    # Select topics based on top_n and topics args
    freq_df = topic_freq
    #freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        topics = sorted(freq_df.Topic.to_list())

    # Select embeddings
    #all_topics = sorted(list(get_topic.keys()))
    #indices = np.array([all_topics.index(topic) for topic in topics])
    #embeddings = embeds[indices]

    # Create dendogram
    distance_matrix = distance_matrix
    fig = ff.create_dendrogram(distance_matrix,
                               orientation=orientation,
                               linkagefun=lambda x: linkage(x, "ward"),
                               color_threshold=1)

    # Create nicer labels
    axis = "yaxis" if orientation == "left" else "xaxis"
    new_labels = [[[str(topics[int(x)]), None]] + get_topic[topics[int(x)]]
                  for x in fig.layout[axis]["ticktext"]]
    new_labels = ["_".join([label[0] for label in labels[:4]]) for labels in new_labels]
    new_labels = [label if len(label) < 30 else label[:27] + "..." for label in new_labels]

    # Stylize layout
    fig.update_layout(
        plot_bgcolor='#ECEFF1',
        template="plotly_white",
        title={
            'text': "<b>Hierarchical Clustering",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
    )

    # Stylize orientation
    if orientation == "left":
        fig.update_layout(height=200+(15*len(topics)),
                          width=width,
                          yaxis=dict(tickmode="array",
                                     ticktext=new_labels))
        
        # Fix empty space on the bottom of the graph
        y_max = max([trace['y'].max()+5 for trace in fig['data']])
        y_min = min([trace['y'].min()-5 for trace in fig['data']])
        fig.update_layout(yaxis=dict(range=[y_min, y_max]))

    else:
        fig.update_layout(width=200+(15*len(topics)),
                          height=height,
                          xaxis=dict(tickmode="array",
                                     ticktext=new_labels))
    return fig


#Getting necessary Data for Viz of all dir_1 Dics

In [14]:
import pickle

In [46]:
dir_1 = ["General, financial and institutional matters",
         "Customs Union and free movement of goods",
         "Agriculture",
         #"Fisheries",
         #"Freedom of movement for workers and social policy",
         #"Right of establishment and freedom to provide services",
         "Transport policy",
         "Competition policy", 
         "Taxation",
         #"Economic and monetary policy and free movement of capital",
         "External relations",
         #"Energy",
         "Industrial policy and internal market",
         "Regional policy and coordination of structural instruments",
         "Environment, consumers and health protection",
         #"Science, information, education and culture",
         #"Law relating to undertakings"
         #"Common Foreign and Security Policy",
         #"Area of freedom, security and justice"
         #"People's Europe" 
         ]

In [15]:
#Function to creat viz_data

def get_viz_data(data, dir, directory):
  topics = []
  embeddings = []
  distances = []

  for name in directory:
    temp = name.split(' ')
    nam = temp[0].replace(',', '')
    embeds = nam+'_embeds'
    dist = nam+'_dist'

    print("* * *")
    print(nam, embeds, dist)

    nombre_1 = embeds
    nombre_2 = dist

    nam, embeds, dist = model_to_figure(data, dir, name)
    
    embed = {f'{nombre_1}': embeds}
    distance = {f'{nombre_2}': dist}

    topics.append(nam)
    embeddings.append(embed)
    distances.append(distance)

  return pd.DataFrame(topics), embeddings, distances

In [ ]:
topics_dir1_df, embeddings_lst, distances_lst = get_viz_data(data, 1, dir_1)

In [131]:
topics_dir1_df = pd.read_pickle('/content/drive/MyDrive/topics_dir1_df.pkl')

In [48]:
topics_dir1_df

,Sub_dir_name,get_topic,topic_freq,topic_sizes
0,"General, financial and institutional matters","{-1: [('shall', 0.09961275711679238), ('articl...",Topic Count 0 -1 60,{-1: 60}
1,Customs Union and free movement of goods,"{0: [('quota', 0.06214412209370968), ('regulat...",Topic Count 0 0 70 1 1 ...,"{0: 70, 1: 67, 2: 39, 3: 35, 4: 31, 5: 28, 6: ..."
2,Agriculture,"{-1: [('shall', 0.028043583855349528), ('membe...",Topic Count 0 0 370 1 1 ...,"{0: 370, 1: 134, 2: 105, -1: 103, 3: 100, 4: 5..."
3,Transport policy,"{-1: [('vehicle', 0.12225904954308939), ('auth...",Topic Count 0 0 37 1 1 3...,"{0: 37, 1: 35, 2: 27, -1: 5}"
4,Competition policy,"{-1: [('aid', 0.13680738970737275), ('shall', ...",Topic Count 0 -1 13,{-1: 13}
5,Taxation,"{-1: [('article', 0.07036896025503037), ('stat...",Topic Count 0 -1 22,{-1: 22}
6,External relations,"{-1: [('footwear', 0.09411896872414789), ('ltd...",Topic Count 0 0 259 1 1 10...,"{0: 259, 1: 101, 2: 24, 3: 14, -1: 7}"
7,Industrial policy and internal market,"{-1: [('insecticide', 0.15599193251816026), ('...",Topic Count 0 0 135 1 1 9...,"{0: 135, 1: 91, 2: 71, 3: 23, 4: 22, 5: 17, 6:..."
8,Regional policy and coordination of structural...,"{-1: [('shall', 0.08791135767436098), ('progra...",Topic Count 0 -1 26,{-1: 26}
9,"Environment, consumers and health protection","{-1: [('ex', 0.060029767201814334), ('name', 0...",Topic Count 0 0 213 1 -1 ...,"{0: 213, -1: 157, 1: 84, 2: 75, 3: 71, 4: 63, ..."


In [171]:
with open('/content/drive/MyDrive/embeddings_dir1.pkl', 'rb') as handle:
    embeddings_lst = pickle.load(handle)

In [135]:
with open('/content/drive/MyDrive/distances_dir1.pkl', 'rb') as handle:
    distances_lst = pickle.load(handle)

In [49]:
bert_bar(topics_dir1_df.iloc[1].topic_freq, topics_dir1_df.iloc[1].get_topic)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [50]:
visualize_topics(topics_dir1_df.iloc[1].topic_freq, topics_dir1_df.iloc[1].topic_sizes, topics_dir1_df.iloc[1].get_topic, embeddings_lst[1]['Customs_embeds'])

In [51]:
visualize_hierarchy(topics_dir1_df.iloc[1].topic_freq, topics_dir1_df.iloc[1].get_topic, distances_lst[1]['Customs_dist'])

In [52]:
topics_dir1_df.to_pickle('/content/drive/MyDrive/topics_dir1_df.pkl')

In [53]:
with open('/content/drive/MyDrive/embeddings_dir1.pkl', 'wb') as handle:
    pickle.dump(embeddings_lst, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
with open('/content/drive/MyDrive/distances_dir1.pkl', 'wb') as handle:
    pickle.dump(distances_lst, handle, protocol=pickle.HIGHEST_PROTOCOL)

##Getting necessary Data for Viz of all *dir_2* Dics

In [16]:
overview_df = pd.read_csv('/content/drive/MyDrive/over_2800_docs_scraped.csv')

In [ ]:
overview_df.head()

In [ ]:
overview_df.dir_2.unique()

In [32]:
dir_2 = [#'Sectoral application', 
         #'Inland transport', 
         'Commercial policy',
         'Social policy', 
         'Health protection', 
         #'Environment',
         'Financial and budgetary provisions', 
         #'Economic policy',
         'Shipping', 
         #'Transport infrastructure',
         #'Economic and social cohesion fund', 
         #'Trans-European networks',
       'Development policy',
       #'General principles', 
       #'programmes and statistics',
       'Industrial policy: sectoral operations', 
       #'Education and training',
       'Industrial policy: general, programmes, statistics and research',
       'General', 
       'Provisions governing the institutions',
       #'European citizenship', 
       'Application of the Common Customs Tariff',
       #'Common fisheries policy',
       #'Products subject to market organisation',
       'Approximation of laws and health measures', 
       #'Basic provisions',
       'Agricultural structures', 
       'Consumers', 
       'Air transport',
       #'Basic customs instruments',
       #'Principles, objectives and tasks of the Treaties',
       'Agricultural structural funds',
       #'Action in favour of countries in transition',
       'Dissemination of information', 
       #'Free movement of persons',
       'Police and judicial cooperation in criminal and customs matters',
       'Judicial cooperation in civil matters',
       #'Internal market: policy relating to undertakings', 
       #'Culture',
       'General principles and programmes', 
       #'Intellectual property law',
       'External relations', 
       'Internal market: approximation of laws',
       'Statistics', 
       #'Monetary measures',
       'Coordination of structural instruments',
       'Bilateral agreements with non-member countries', 
       'Programmes',
       #'Company law', 
       #'Specific customs rules', 
       'General customs rules',
       #'Indirect taxation', 
       #'Science', 
       #'Free movement of capital',
       #'Prevention of tax evasion and avoidance', 
       #'Monetary policy',
       #'Other sources of energy', 
       #'Multilateral relations',
       #'Nuclear energy', 
       #'Protection of animals',
       #'State aids and other subsidies', 
       #'Competition principles',
       #'Public contracts', 
       #'Principles and conditions', 
       'Electricity',
       #'Restrictive practices',
       #'European Regional Development Fund (ERDF)',
       #'Freedom of movement for workers', 
       #'Oil and gas',
       #'Products not subject to market organisation', 
       #'Economic and commercial law'
       ]

In [ ]:
topics_dir2_df, embeddings2_lst, distances2_lst = get_viz_data(data, 2, dir_2)

In [34]:
topics_dir2_df.to_pickle('/content/drive/MyDrive/topics_dir2_df.pkl')

In [37]:
with open('/content/drive/MyDrive/embeddings_dir2.pkl', 'wb') as handle:
    pickle.dump(embeddings2_lst, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
with open('/content/drive/MyDrive/distances_dir2.pkl', 'wb') as handle:
    pickle.dump(distances2_lst, handle, protocol=pickle.HIGHEST_PROTOCOL)

##Getting necessary Data for Viz of all *dir_3* Dics

In [17]:
overview_df.dir_3.unique()

array(['Service activities', 'Market operation', 'Trade protection',
       'Employment and unemployment', nan, 'Pollution and nuisances',
       'Instruments of economic policy', 'Financial support',
       'European Social Fund (ESF)', 'General',
       'Information technology, telecommunications and data-processing',
       'Research and technological development',
       'General provisions and programmes', 'Economic and monetary union',
       'Programmes and statistics', 'Tariff derogations',
       'Conservation of resources', 'Sheepmeat and goatmeat',
       'Beef and veal', 'Animal health and zootechnics',
       'Common agricultural policy mechanisms', 'Safety at sea',
       'Social and structural measures',
       'Consumer information, education and representation', 'Air safety',
       'Milk products', 'Plant health', 'Animal feedingstuffs',
       'Origin of goods', 'Generalised system of preferences',
       'Trade arrangements', 'Protection of health and safety',
     

In [147]:
dir_3 = [#'Service activities', 
         'Market operation', 
         #'Trade protection',
         #'Employment and unemployment', 
         'Pollution and nuisances',
       'Instruments of economic policy', 
       #'Financial support',
       #'European Social Fund (ESF)', 
       'General',
       'Information technology, telecommunications and data-processing',
       #'Research and technological development',
       #'General provisions and programmes', 
       #'Economic and monetary union',
       #'Programmes and statistics', 
       #'Tariff derogations',
       #'Conservation of resources', 
       #'Sheepmeat and goatmeat',
       #'Beef and veal', 
       #'Animal health and zootechnics',
       #'Common agricultural policy mechanisms', 
       #'Safety at sea',
       'Social and structural measures',
       'Consumer information, education and representation', 
       'Air safety',
       'Milk products', 
       'Plant health', 
       #'Animal feedingstuffs',
       'Origin of goods', 
       'Generalised system of preferences',
       'Trade arrangements', 
       #'Protection of health and safety',
       #'Working conditions', 
       'Structural measures',
       #'European Agricultural Fund for Rural Development',
       'European Agricultural Guarantee Fund',
       #'Immigration and the right of nationals of third countries',
       'Crossing external borders', 
       'Market organisation',
       'Rational utilisation and conservation of energy',
       #'Multilateral relations', 
       'Proprietary medicinal products',
       'Motor vehicles', 
       #'Foodstuffs', 
       'Protection of economic interests',
       'Tariff classification', 
       #'Customs tariffs', 
       #'Cereals', 
       'Wine',
       #'Rice', 
       #'Eggs and poultry', 
       #'Structural harmonisation',
       'General social provisions', 
       'Other commercial policy measures',
       #'European countries', 
       #'Judicial cooperation in criminal matters',
       #'Asylum policy', 
       #'Movement of goods', 
       'Fresh fruit and vegetables',
       'Common customs territory',
       #'Other sectors for approximation of laws', 
       #'Turnover tax/VAT',
       #'Dried fodder', 
       'Oils and fats',
       'Arrangements covering more than one market organisation',
       'Dangerous substances', 
       #'Own resources', 
       #'Research sectors',
       #'Elimination of internal border controls',
       'Space, environment and natural resources', 
       #'Customs cooperation',
       #'Institutional monetary provisions',
       'Institutional economic provisions',
       #'African, Caribbean and Pacific (ACP) Group of States',
       'Agreements with non-member countries',
       #'Indirect instruments of monetary policy',
       #'Approximation of certain social provisions',
       #'General, programmes', 
       #'Other spheres of multilateral cooperation',
       #'Seeds and seedlings', 
       #'Nuclear research', 
       #'Excise duties', 
       #'Hops',
       #'National aid', 
       'Agricultural and forestry tractors', 
       #'Cosmetics',
       #'Accountancy data network', 
       #'Fertilisers',
       #'Multilateral cooperation for protection of the environment, wild fauna and flora and natural resources',
       #'Supervision procedures', 
       #'Financial and economic Aid',
       #'User tariffs', 
       #'Budget', 
       #'Raw tobacco', 
       #'Police cooperation',
       #'Supplies and stocks',
       #'Products processed from fruit and vegetables',
       #'Aid to developing countries', 
       #'Peas and beans',
       #'Other measures relating to nuclear energy',
       #'Other economic and commercial provisions', 
       'Sugar'
       #'Court of Justice'
       ]

In [146]:
test, test_emb, test_dis = model_to_figure(data, 3, "Court of Justice")

TypeError: ignored

In [148]:
topics_dir3_df, embeddings3_lst, distances3_lst = get_viz_data(data, 3, dir_3)

* * *
Market Market_embeds Market_dist


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-08 08:53:39,923 - BERTopic - Transformed documents to Embeddings
2022-06-08 08:53:42,537 - BERTopic - Reduced dimensionality
2022-06-08 08:53:42,555 - BERTopic - Clustered reduced embeddings


* * *
Pollution Pollution_embeds Pollution_dist


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-08 08:59:21,193 - BERTopic - Transformed documents to Embeddings
2022-06-08 08:59:23,598 - BERTopic - Reduced dimensionality
2022-06-08 08:59:23,608 - BERTopic - Clustered reduced embeddings


* * *
Sugar Sugar_embeds Sugar_dist


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-08 08:59:29,646 - BERTopic - Transformed documents to Embeddings
2022-06-08 08:59:32,043 - BERTopic - Reduced dimensionality
2022-06-08 08:59:32,052 - BERTopic - Clustered reduced embeddings


In [149]:
topics_dir3_df.to_pickle('/content/drive/MyDrive/topics_dir3_df.pkl')

In [150]:
with open('/content/drive/MyDrive/embeddings_dir3.pkl', 'wb') as handle:
    pickle.dump(embeddings3_lst, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [151]:
with open('/content/drive/MyDrive/distances_dir3.pkl', 'wb') as handle:
    pickle.dump(distances3_lst, handle, protocol=pickle.HIGHEST_PROTOCOL)